In [1]:
import os
os.environ['KERAS_BACKEND'] = 'torch'

In [2]:
from nltk.corpus import movie_reviews

from sklearn.model_selection import train_test_split

import numpy as np

import tensorflow as tf
import torch

import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding, TextVectorization

In [3]:
fileids = movie_reviews.fileids()
reviews = [movie_reviews.raw(fileid) for fileid in fileids]
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids]

In [4]:
# 1. 백엔드 이름 확인
print(f"현재 Keras 백엔드: {keras.backend.backend()}")

# 2. GPU 사용 가능 여부 확인 (PyTorch 방식)
print(f"GPU 사용 가능: {torch.cuda.is_available()}")

# 3. 간단한 Keras 모델 생성 및 장치 확인
model = keras.Sequential([keras.layers.Dense(10, input_shape=(5,))])
# Keras 텐서가 어떤 장치에 있는지 확인 (PyTorch 백엔드일 경우)
print(f"모델 파라미터가 위치한 장치: {next(model.parameters()).device}")

현재 Keras 백엔드: torch
GPU 사용 가능: True
모델 파라미터가 위치한 장치: cuda:0


c:\Coding\Local\python_text_mining\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
np.random.seed(7)
tf.random.set_seed(7)

In [6]:
max_tokens = 10000
# torch로 keras의 backend를 변경하면서 방식 변경
# tokenizer뿐만 아니라 padding, truncation 까지 이 단계에서 수행
tokenizer = TextVectorization(max_tokens=max_tokens, output_sequence_length=1000)
tokenizer.adapt(reviews)
X = tokenizer(reviews)
print([len(doc) for doc in X[:10]])

[1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]


In [7]:
vocab = tokenizer.get_vocabulary()
word_index = {word: index for index, word in enumerate(vocab)}
print(word_index['the'])
print(word_index['review'])

2
457


In [8]:
label_dict = {'pos':1, 'neg':0}
y = np.array([label_dict[c] for c in categories])
print(set(y))

{np.int64(0), np.int64(1)}


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)
print(X_train.shape, X_test.shape)

torch.Size([1600, 1000]) torch.Size([400, 1000])
